In [1]:
import pandas as pd, numpy as np

In [2]:
def HistSquareDiff(exp, ctrl, factor=1):
    """The actual workhorse HistDiff scoring function"""

    # we transpose twice to ensure the final result is a 1 x m feature score vector
    ctrl_meanProxy = (np.arange(1, ctrl.shape[0] + 1) * ctrl.T).T.sum(axis=0)
    exp_meanProxy = (np.arange(1, exp.shape[0] + 1) * exp.T).T.sum(axis=0)

    # evaluate where and when to adjust the score to be negative
    negScore = np.where(ctrl_meanProxy > exp_meanProxy, -1, 1)
    diff = ctrl - (exp.T * factor)
    diff **= 2

    return diff.sum(axis=1) * negScore


In [4]:
exp_mat = np.array([[1, 2],
                    [2, 3],
                    [3, 4]])
crt_mat = np.array([2,3,4])

HistSquareDiff(exp=exp_mat, ctrl=crt_mat, factor=0.5)

array([-12.5 ,   7.25])